In [263]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [269]:
%%sql
DROP TABLE if exists Products;
CREATE TABLE Products
(
    id integer primary key,
    name VARCHAR(16) NOT NULL,
    price INTEGER NOT NULL
);


--重複するレコード
INSERT INTO Products VALUES(1,'りんご',	50);
INSERT INTO Products VALUES(2,'みかん',	100);
INSERT INTO Products VALUES(3,'みかん',	80);
INSERT INTO Products VALUES(4,'みかん',	110);
INSERT INTO Products VALUES(5,'バナナ',	80);
INSERT INTO Products VALUES(6,'バナナ',	60);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [272]:
%%sql
-- window関数を使う方法
select *,
    row_number() over (
        partition by name
        order by price
    ) as seq
from Products
order by name, price

*  postgresql://padawan:***@db:5432/dsdojo_db
6 rows affected.


id,name,price,seq
3,みかん,80,1
2,みかん,100,2
4,みかん,110,3
1,りんご,50,1
6,バナナ,60,1
5,バナナ,80,2


In [273]:
%%sql
-- 相関サブクエリを使う方法
select *, (
    select count(*)
    from Products as p2
    where p1.name = p2.name and
    p1.price >= p2.price
) as seq
from Products as p1
order by name, price

*  postgresql://padawan:***@db:5432/dsdojo_db
6 rows affected.


id,name,price,seq
3,みかん,80,1
2,みかん,100,2
4,みかん,110,3
1,りんご,50,1
6,バナナ,60,1
5,バナナ,80,2
